# Part 3 - Visualising Particle Tracks

When running neutronics simulations we may want to track how particles travel through the defined geometry.

This python notebook allows users to generate particle track files that can be opened and viewed alongside the 3D geometry.

This first code block makes a geometry with hemispheres. One side is a moderator (H2O) and the other is a material that is quite transparent to neutrons (zirconium).

Because of the different neutronic properties of the materials, we expect neutrons to track differently through the two materials. We can visualise this using OpenMC.

In [1]:
import openmc
import matplotlib.pyplot as plt
import os

# MATERIALS
mats = openmc.Materials()

moderating_material = openmc.Material(1, "water")  # water contains hydrogen which is a good neutron moderator
moderating_material.add_element('H', 2, 'ao')  # Note, 'percent_type=' does not have to be written to specify 'ao' or 'wo'
moderating_material.add_element('O', 1, 'ao')
moderating_material.set_density('g/cm3', 1.0)
mats.append(moderating_material)

transparent_material = openmc.Material(2, "zirconium")  # one of the more transparent materials
transparent_material.add_element('Zr', 1, 'ao')
transparent_material.set_density('g/cm3', 6.49)
mats.append(transparent_material)


# GEOMETRY
sph0 = openmc.Sphere(r=400)
sph1 = openmc.Sphere(r=600, boundary_type='vacuum')
flat_surf = openmc.YPlane(y0=0)

simple_moderator_cell = openmc.Cell(region=+sph0 & -sph1 & +flat_surf)
simple_moderator_cell.fill = moderating_material

simple_transparent_cell = openmc.Cell(region=+sph0 & -sph1 & -flat_surf)
simple_transparent_cell.fill = transparent_material

vaccum_cell = openmc.Cell(region=-sph0)

universe = openmc.Universe(cells=[simple_moderator_cell, simple_transparent_cell, vaccum_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
batches = 1
sett.batches = batches
sett.inactive = 0
sett.particles = 10  # Note that only 10 particles are simulated, otherwise we make too many files
sett.particle = "neutron"
sett.track = (1, 2, 4)
sett.run_mode = 'fixed source'

# creates a 14MeV point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])

# source.file = 'source_1000_particles.h5'
sett.source = source

This is the new part covered by this task. The running of OpenMC in track mode.

In [2]:
# Run OpenMC!
model = openmc.model.Model(geom, mats, sett)
model.run(tracks=True)  # this creates h5 files that contain track information

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/jshim/neutronics-workshop/tasks/task_04_make_sources/statepoint.1.h5')

This loops through the h5 files and uses a script provided with OpenMC to convert them to vtp files (which can be opened with Paraview).

In [3]:
for i in range(1, 11):
    os.system('openmc-track-to-vtk track_1_1_'+str(i)+'.h5 -o track_1_1_'+str(i))

This next code block might be familar from task 3. Here, we are converting the geometry into a 3D version which can be viewed. This will provide a nice background for the tracks so we know where are relative to the materials.

In [4]:
vox_plot = openmc.Plot()
vox_plot.type = 'voxel'
vox_plot.width = (1300., 1300., 1300.)
vox_plot.pixels = (100, 100, 100)
vox_plot.filename = 'plot_3d'
vox_plot.color_by = 'material'
vox_plot.colors = {moderating_material: 'blue', transparent_material: 'red'}
plots = openmc.Plots([vox_plot])
plots.export_to_xml()

openmc.plot_geometry()
# openmc-voxel-to-vtk is a python script distributed with openmc
os.system('openmc-voxel-to-vtk plot_3d.h5 -o plot_3d.vti')

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

0

This next command generates a download link to the 3D vti and vtp files generated.

Click on the link to download the vtk file onto your base computer and open it with a VTK file reader such as Paraview or Visit.

Paraview can be downloaded here: https://www.paraview.org/download/. Visit can be downloaded here https://wci.llnl.gov/simulation/computer-codes/visit/downloads.


In [5]:
from IPython.display import FileLink
display(FileLink('plot_3d.vti'))
display(FileLink('track_1_1_1_0.vtp'))
display(FileLink('track_1_1_2_0.vtp'))
display(FileLink('track_1_1_3_0.vtp'))
display(FileLink('track_1_1_4_0.vtp'))
display(FileLink('track_1_1_5_0.vtp'))
display(FileLink('track_1_1_6_0.vtp'))
display(FileLink('track_1_1_7_0.vtp'))
display(FileLink('track_1_1_8_0.vtp'))
display(FileLink('track_1_1_9_0.vtp'))
display(FileLink('track_1_1_10_0.vtp'))

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/plot_3d.vti

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_1_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_2_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_3_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_4_0.vtp

...

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_6_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_7_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_8_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_9_0.vtp

/home/jshim/neutronics-workshop/tasks/task_04_make_sources/track_1_1_10_0.vtp

**Learning Outcomes for Part 3:**

- Particle tracks through geometry can be tracked in OpenMC and visualised in programs such as Paraview and Visit.